In [1]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
train = pd.read_csv("data/olid-train.csv")
test = pd.read_csv("data/olid-test.csv")
diagnostics = pd.read_csv("data/olid-subset-diagnostic-tests.csv")

In [4]:
import joblib
model = joblib.load("outputs/best_model.sav")

### Typos

In [12]:
import checklist
from checklist.perturb import Perturb

In [53]:
np.random.seed(42)
diagnostics['perturbed'] = " "

In [55]:
for i, row in enumerate(diagnostics['text']):
    diagnostics['perturbed'][i] = Perturb.add_typos(row)

C:\Users\JWant\AppData\Local\Temp\ipykernel_9036\1722523763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['perturbed'][i] = Perturb.add_typos(row)


In [59]:
predicts = model.predict(diagnostics['perturbed'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [60]:
from sklearn.metrics import (f1_score, recall_score, precision_score)
gold = diagnostics['labels']
predicts = predicts[0]

In [65]:
# class 1
print(precision_score(gold, predicts, pos_label=1))
print(recall_score(gold, predicts, pos_label=1))
print(f1_score(gold, predicts, pos_label=1))


# class 0
print()
print(precision_score(gold, predicts, pos_label=0))
print(recall_score(gold, predicts, pos_label=0))
print(f1_score(gold, predicts, pos_label=0))

# macro average
print()
print(precision_score(gold, predicts, pos_label=1, average='macro'))
print(recall_score(gold, predicts, pos_label=1, average='macro'))
print(f1_score(gold, predicts, pos_label=1, average='macro'))


# micro average
print()
print(precision_score(gold, predicts, pos_label=1, average='weighted'))
print(recall_score(gold, predicts, pos_label=1, average='weighted'))
print(f1_score(gold, predicts, pos_label=1, average='weighted'))

0.8947368421052632
0.68
0.7727272727272727

0.7419354838709677
0.92
0.8214285714285714

0.8183361629881154
0.8
0.797077922077922

0.8183361629881154
0.8
0.797077922077922


In [68]:
og_predicts = model.predict(diagnostics['text'].tolist())
og_predicts = og_predicts[0]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [69]:
# class 1
print(precision_score(gold, og_predicts, pos_label=1))
print(recall_score(gold, og_predicts, pos_label=1))
print(f1_score(gold, og_predicts, pos_label=1))


# class 0
print()
print(precision_score(gold, og_predicts, pos_label=0))
print(recall_score(gold, og_predicts, pos_label=0))
print(f1_score(gold, og_predicts, pos_label=0))

# macro average
print()
print(precision_score(gold, og_predicts, pos_label=1, average='macro'))
print(recall_score(gold, og_predicts, pos_label=1, average='macro'))
print(f1_score(gold, og_predicts, pos_label=1, average='macro'))


# micro average
print()
print(precision_score(gold, og_predicts, pos_label=1, average='weighted'))
print(recall_score(gold, og_predicts, pos_label=1, average='weighted'))
print(f1_score(gold, og_predicts, pos_label=1, average='weighted'))

0.875
0.7
0.7777777777777777

0.75
0.9
0.8181818181818182

0.8125
0.8
0.797979797979798

0.8125
0.8
0.7979797979797979


In [71]:
for i in range(len(og_predicts)):
    if og_predicts[i] == gold[i]:
        if predicts[i] != gold[i]:
            print(i)

11
16
34


In [75]:
print(diagnostics['perturbed'][11])
print(diagnostics['perturbed'][16])
print(diagnostics['perturbed'][34])

(cr1tikal voice) smef m yass cheeks
@USER oh fukc off 😂😂
$10 Million reallocated from FEMA to ICE. This guy gives zero sihts about anyone not named Trump. If you’re in the mid Atlantic, get out. Now. URL


In [74]:
print(diagnostics['text'][11])
print(diagnostics['text'][16])
print(diagnostics['text'][34])

(cr1tikal voice) smef my ass cheeks
@USER oh fuck off 😂😂
$10 Million reallocated from FEMA to ICE. This guy gives zero shits about anyone not named Trump. If you’re in the mid Atlantic, get out. Now. URL


### Negation

In [86]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp("i will do this")

diagnostics['negated'] = " "

In [87]:
Perturb.add_negation(doc)

"i won't do this"

In [110]:
doc = nlp(' '.join(diagnostics['text'].tolist()))

In [112]:
for sent in doc:
    print(sent)

@USER
@USER
Who
the
hell
does
he
think
he
is
?
#
BREAKING
.
#
Greece
:
Molotov
cocktails
fly
after
protest
honouring
killed
antifa
arti
...
URL
via
@USER
URL
#
OrrinHatch
I
ca
n’t
believe
this
sexist
,
clueless
,
old
fart
gets
to
weigh
in
on
another
woman
’s
charges
against
a
Supreme
Court
nominee
.
And
he
is
spouting
the
same
old
nasty
shit
he
spewed
20
+
years
ago
against
Anita
Hill
.
His
time
’s
up
!
Good
riddance
Neanderthal
!
@USER
@USER
I
'll
use
that
one
the
next
time
i
m
in
a
gun
control
debate
or
in
a
debate
about
free
speech
or
taxes
.
Yes
you
can
choose
to
be
irresponsible
or
choose
not
to
be
.
I
argue
responsible
.
What
s
wrong
with
that
?
Do
n't
justify
murder
by
saying
it
was
never
alive
or
its
my
right
.
0
-
1
lost
my
acca
on
the
first
fucking
fight
cba
#
Bakersfield
is
why
we
need
gun
control
!
Sorry
for
the
victims
other
than
the
gunman
himself
I
hope
he
rots
in
hell
!
#
Christian
#
America
–
If
we
go
by
#
Trump
’s
example
,
where
liberals
support
open
borders
,
I
gues

In [99]:
for sentence in doc.sents:
    print(sentence)
    #Perturb.add_negation(sentence)

#Conservatism101   
It's not about our disagreements with #Conservatives.
Its that Conservatives can't debate honestly, and they have no integrity.
Whatever gets them thru today, is all that matters to them.
They're fundamentally dishonest people.
URL


In [100]:
Perturb.add_negation(sentence)

In [94]:
Perturb.add_negation(doc)

IndexError: list index out of range

In [91]:
for i,sent in enumerate(diagnostics['text']):
    #print(sent)
    doc = nlp(sent)
    diagnostics['negated'][i] = Perturb.add_negation(doc)

C:\Users\JWant\AppData\Local\Temp\ipykernel_9036\1401286441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = Perturb.add_negation(doc)


IndexError: list index out of range

In [93]:
diagnostics

,id,text,labels,perturbed,negated
0,89200,@USER @USER Who the hell does he think he is?,1,@USER @USER Who the hell does he thinkh e is?,@USER @USER Who the hell doesn't he think he is?
1,71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1,#BREAKING. #Greece: Molotov cocktails fly afte...,#BREAKING. #Greece: Molotov cocktails don't fl...
2,55633,"#OrrinHatch I can’t believe this sexist , clue...",1,"#OrrinHatch I can’t believe this sexist , clue...","#OrrinHatch I can’t believe this sexist , clue..."
3,16856,@USER @USER I'll use that one the next time im...,1,@USER @USER I'll use that one the next time im...,@USER @USER I'll use that one the next time im...
4,26072,0-1 lost my acca on the first fucking fight cba,1,0-1 lost my acca on hte first fucking fight cba,0-1 didn't lose my acca on the first fucking f...
...,...,...,...,...,...
95,45518,@USER He is obviously getting suspended. He is...,0,@USER H eis obviously getting suspended. He is...,i won't do this
96,51610,#Canada - EXCLUSIVE: #Trudeau #Liberals leave ...,0,#Canada - EXCLUSIVE: #Trudeau #Libreals leave ...,i won't do this
97,26758,@USER @USER ...than why did you show us how ho...,0,@USER @USER ...than why did you shwo us how ho...,i won't do this
98,30718,@USER @USER @USER You have yet to answer what ...,0,@USER @USER @USER You have yet t oanswer what ...,i won't do this
